In [10]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import os

In [39]:
PATH = '/Users/ansat.omurzakov/Desktop/TUe/Data Challenge 2/data/'
PATH_to_neighbors = '/Users/ansat.omurzakov/Desktop/TUe/Data Challenge 2/neighbourhood/'

In [131]:
def delete_files(PATH):
    # getting dates from the CSV file
    dates_list = os.listdir(PATH)
    for date in dates_list:
        if not re.match(r'\d{4}-\d{2}', date):
            dates_list.remove(date)

    dates_list = sorted(dates_list)

    # deleting irrelevant files from directories
    for date in tqdm(dates_list, total = len(dates_list)):
        for file in os.listdir(PATH + date):
            if not re.search(r'(btp)|(metropolitan)', file):
                os.remove(PATH + '/' + date + '/' + file)
    return 'Done!'

def save_data(category, PATH):
    if len(category) != 0:
        all_data = pd.DataFrame()
        for f in tqdm(category, total = len(category)):
            df = pd.read_csv(PATH + f[:7] + '/' + f)
            all_data = pd.concat([all_data,df],ignore_index=True)
        all_data.to_csv(PATH + re.split(r'(\d{2}-)', category[0])[-1], index=False)
    else: return f'List {category} is empty'

def get_file_list(department, type, PATH):
    lst = []
    for _, _, files in os.walk(PATH, topdown = True):
        for filename in files:
            if f'{department}-{type}' in filename:
                lst.append(filename)
    return lst

## Deleting irrelevant files from directories (neighborhood, data)

In [ ]:
delete_files(PATH)
delete_files(PATH_to_neighbors)

## Getting file names from which to retrieve csvs

In [129]:
# Metropolitan_files
metropolitan_street = get_file_list('metropolitan','street', PATH)
metropolitan_outcomes= get_file_list('metropolitan','outcomes', PATH)
metropolitan_sas= get_file_list('metropolitan','stop-and-search', PATH)
metropolitan_neighborhood= get_file_list('metropolitan','neighbourhood', PATH_to_neighbors)

# BTP files
btp_street = get_file_list('btp','street', PATH)
btp_outcomes= get_file_list('btp','outcomes', PATH)
btp_sas= get_file_list('btp','stop-and-search', PATH)

## Concatenating files per category and saving them into one csv file

In [130]:
# Metropolitan Data
save_data(metropolitan_street, PATH)
save_data(metropolitan_outcomes, PATH)
save_data(metropolitan_sas, PATH)
save_data(metropolitan_neighborhood, PATH_to_neighbors)

# BTP data
save_data(btp_street, PATH)
save_data(btp_outcomes, PATH)
save_data(btp_sas, PATH)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/143 [00:00<?, ?it/s]

  0%|          | 0/106 [00:00<?, ?it/s]

# Getting all neighbourhoods

In [1]:
import requests

In [2]:
locate_neighbourhood_link = 'https://data.police.uk/api/metropolitan/neighbourhoods'
met_neighbourhoods = requests.get(locate_neighbourhood_link).json()

In [3]:
neighbourhoods = [neighbourhood['id'] for neighbourhood in met_neighbourhoods]

### Classify per borough

In [4]:
from tqdm.autonotebook import tqdm

/var/folders/9z/6v7l930d5d3fnhz8w_cp8_gw0000gn/T/ipykernel_15105/987820437.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
import re

In [7]:
dick = {}
for value in tqdm(neighbourhoods, total = len(neighbourhoods)):
    try:
        data = requests.get(f'https://findthatpostcode.uk/areas/{value}.json').json()['included']
        name = data[6]['attributes']['name']
        if name in dick.keys():
            dick[name].append(value)
        else:
            dick[name] = [value]
    except Exception as e:
        data = requests.get(f'https://findthatpostcode.uk/areas/{value[:-1]}.json').json()['included']
        name = data[6]['attributes']['name']
        if name in dick.keys():
            dick[name].append(value)
        else:
            dick[name] = [value]
print('Done')

In [ ]:
import json
with open(PATH + 'boroughs_neighbourhoods.json', 'w') as file:
    json.dump(dick, file)

In [4]:
import json
import pandas as pd

In [5]:
# Load JSON data from file
with open('/Users/ansat.omurzakov/Desktop/TUe/Data Challenge 2/data/boroughs_neighbourhoods.json', 'r') as file:
    json_data = json.load(file)

# Convert JSON data to a DataFrame
boroughs_data = []
for borough, area_codes in json_data.items():
    for area_code in area_codes:
        boroughs_data.append({'Borough': borough, 'Area Code': area_code})

boroughs_neighbours = pd.DataFrame(boroughs_data)
boroughs_neighbours

,Borough,Area Code
0,Tower Hamlets,E05009317
1,Tower Hamlets,E05009318
2,Tower Hamlets,E05009319
3,Tower Hamlets,E05009320
4,Tower Hamlets,E05009321
...,...,...
674,Lambeth,E05014115
675,Lambeth,E05014116
676,Lambeth,E05014117
677,Lambeth,E05014118


In [13]:
import requests

In [23]:
import requests
from tqdm import tqdm

b_n_b = {}

for borough in tqdm(boroughs_neighbours['Borough'].unique(), total=len(boroughs_neighbours['Borough'].unique())):
    if borough not in b_n_b:
        b_n_b[borough] = []
    for neighbourhood in boroughs_neighbours[boroughs_neighbours['Borough'] == borough]['Area Code'].unique():
        boundary_points = [(item['latitude'], item['longitude']) for item in requests.get(f'https://data.police.uk/api/metropolitan/{neighbourhood}/boundary').json()]
        b_n_b[borough].append({neighbourhood: boundary_points})
    print(f'[INFO] Added neighbourhoods and their boundaries for {borough}')

  3%|▎         | 1/32 [00:07<03:52,  7.50s/it]

[INFO] Added neighbourhoods and their boundaries for Tower Hamlets


  6%|▋         | 2/32 [00:15<03:53,  7.79s/it]

[INFO] Added neighbourhoods and their boundaries for Hackney


  9%|▉         | 3/32 [00:22<03:33,  7.36s/it]

...

[INFO] Added neighbourhoods and their boundaries for Barking and Dagenham


 97%|█████████▋| 31/32 [04:19<00:08,  8.36s/it]

[INFO] Added neighbourhoods and their boundaries for Greenwich


100%|██████████| 32/32 [04:30<00:00,  8.46s/it]

[INFO] Added neighbourhoods and their boundaries for Lambeth


In [30]:
# Specify the file path
json_file_path = "/Users/ansat.omurzakov/Desktop/TUe/Data Challenge 2/data/neighborhood_boundaries.json"

# Save the dictionary as a JSON file
with open(json_file_path, "w") as json_file:
    json.dump(b_n_b, json_file)

print("JSON file saved successfully.")

JSON file saved successfully.
